In [24]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.agents.assistant_agent import AssistantAgent

topic = "Calculus"
subject = "Mathematics"
standard = 11

# 1. Instantiate the agent directly
agent = AssistantAgent()

# 2. Use 'await' directly (works only in Jupyter/IPython)
assistant_response = await agent.run(
    topic=topic,
    subject=subject,
    standard=standard
)

In [ ]:
print(assistant_response)

In [26]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.prompts.tutoring_prompt import LESSON_PLANNING_PROMPT
from backend.src.logger import logging
from backend.src.utils import get_llm_lite_1
from langchain_core.messages import SystemMessage
from backend.src.exception import CustomException
import sys

llm_lite = get_llm_lite_1()

async def plan_curriculum(instructions) -> dict:
        try:
            logging.info("Generating curriculum plan...")
            
            prompt = LESSON_PLANNING_PROMPT.format(
                topic = "Calculus",
                subject = "Mathematics",
                standard = 11,
                instructions = instructions
            )

            response = await llm_lite.ainvoke([SystemMessage(content=prompt)])
            
            return {"plannings": response.content}

        except Exception as e:
            raise CustomException(e, sys)

In [27]:
plans = await plan_curriculum(instructions = assistant_response)

In [ ]:
print(plans["plannings"])

In [29]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.prompts.tutoring_prompt import LESSON_CONTENT_PROMPT
from backend.src.utils import get_llm_1
from langchain_core.messages import SystemMessage
from backend.src.logger import logging
from backend.src.exception import CustomException
import asyncio

llm = get_llm_1()

async def generate_parallel_lessons(plans, topic, subject, standard) -> dict:
        try:
            logging.info("Generating lessons with rate limiting...")

            plannings = plans
            topic = topic
            subject = subject
            standard = standard
            
            lesson_list = [line.strip() for line in plannings.split("\n") if line.strip()]

            # --- KEY FIX: SEMAPHORE ---
            # This limits the agent to only 3 concurrent requests at a time.
            # You can increase to 5 if you have a paid/higher tier API plan.
            sem = asyncio.Semaphore(3) 

            async def generate_single_lesson_safe(lesson_title):
                """Generates content for one lesson, respecting the semaphore limit."""
                async with sem:  # Wait for a 'slot' to open before requesting
                    try:
                        # Add a small delay to be extra safe against RPM limits
                        await asyncio.sleep(0.1) 
                        
                        p = LESSON_CONTENT_PROMPT.format(
                            lesson_title=lesson_title,
                            topic=topic,
                            subject=subject,
                            standard=standard
                        )
                        res = await llm.ainvoke([SystemMessage(content=p)])
                        logging.info(f"Generated: {lesson_title[:30]}...")
                        return lesson_title, res.content
                    
                    except Exception as e:
                        # Log the specific error (likely 429)
                        logging.error(f"Failed to generate {lesson_title}: {str(e)}")
                        # Retry logic could be added here, but for now we return error html
                        return lesson_title, f"<p>Error generating content: {str(e)}</p>"

            # Create tasks
            tasks = [generate_single_lesson_safe(lesson) for lesson in lesson_list]

            # Run tasks (They will now respect the limit of 3 at a time)
            results = await asyncio.gather(*tasks)

            lessons_dict = {title: content for title, content in results}

            logging.info(f"Generated {len(lessons_dict)} lessons successfully.")

            return {"lessons": lessons_dict}

        except Exception as e:
            raise CustomException(e, sys)

In [31]:
lessons = await generate_parallel_lessons(
    plans = plans["plannings"],
    topic = "Calculus",
    subject = "Mathematics",
    standard = 11
)

In [ ]:
for title, content in lessons["lessons"].items():

    print(title, "\n")
    print(content)
    print("\n")
    print("-" * 100)